In [93]:
import json
import openai

openai.api_key = 'your_key'
messages = [ {"role": "system", "content": "You are a intelligent assistant."} ]

In [161]:
def chat_with_chatgpt(_messages, prompt, _model="gpt-3.5-turbo"):

# with only the current clip as information
    chat = openai.ChatCompletion.create(
        model=_model, messages= [{"role": "user", "content": prompt}]
    )

#   with all the context in the same video clip $TOO LARGE TOKEN SIZE
#   _messages.append({"role": "user", "content": prompt},)
#   chat = openai.ChatCompletion.create(
#       model=_model, messages=_messages
#   )

    reply = chat.choices[0].message.content
#     _messages.append({"role": "assistant", "content": reply})
    return reply

In [216]:
def generateQA(data_file):
    QA = {}
    
    f = open(data_file)
    
    data = json.load(f)
    for video_id in data:
        messages = [ {"role": "system", "content": 
              "You are a intelligent assistant."} ]
        QA[video_id] = {}
        QA[video_id]['annotation_qa'] = []
        QA[video_id]['summary_qa'] = []
        video = data[video_id]
        
        i = 0
        for item in video['narrations']:
            if i == 20:
                break;
            prompt = get_prompt(' start_time: ' + str(item['time']) 
                                + ' end_time: ' + str(item['_end_time']) 
                                + ' description: ' + str(item['text']))
            QA[video_id]['annotation_qa'].append(chat_with_chatgpt(messages, prompt))
            i += 1
        
        i = 0
        for item in video['summaries']:
            if i == 20:
                break;
            prompt = get_prompt(' start_time: ' + str(item['start_time']) 
                                + ' end_time: ' + str(item['end_time']) 
                                + ' description: ' + str(item['text']))
            QA[video_id]['summary_qa'].append(chat_with_chatgpt(messages, prompt))
            i += 1

    return QA
            

In [217]:
def get_prompt(data_location):
    return """
    You are given the description of a video clip, and the data has a start time, end time, and description of the clip.
    You must generate a set of question answer pairs for this video clip.
    Ask the question "When does (your detailed description of the object) appear?" for every physical object (including things like keys, spoon, and phone but does not including non-physical object such as a conversation or a movement) that appears in the description.
    And the answer must be in the format: start_time: , end_time: .
    If there is no physical object in the description, answer "no objects"
    Here is the format:\n Q:\n A:\n’.
    
    Now complete your task given the start time, end time, and context of the video as the following: %s0
    """ % data_location

In [ ]:
QA = generateQA('out.json')

In [220]:
with open('ego4DQA.json', 'w') as wf:
    json.dump(QA, wf)